In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def digitizeGraph(image):
    # Load the image
    img = cv2.imread(image, cv2.IMREAD_COLOR)

    plt.imshow(img)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


    # Apply Canny edge detection
    edges = cv2.Canny(gray, threshold1=100, threshold2=150, apertureSize=7)

    # Apply Hough Circle Transform
    circles = cv2.HoughCircles(gray,cv2.HOUGH_GRADIENT, dp=1, minDist=20, param1=50, param2=30, minRadius=0, maxRadius=0)


    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=100, maxLineGap=0)



    #lines are given in xy coordinates
    # Determine which circles are connected by lines
    connected_circles = []
    for line_id, line in enumerate(lines):
        x1, y1, x2, y2 = line[0]

        two_circles = []
        for circle_id, circle in enumerate(circles[0]):
            cx, cy, _ = circle
            distance1 = np.sqrt((cx - x1)**2 + (cy - y1)**2)
            distance2 = np.sqrt((cx - x2)**2 + (cy - y2)**2)
            if distance1 <= circles[0][0][2] + 100 or distance2 <= circles[0][0][2] + 100:
                two_circles.append(circle_id + 1)
        connected_circles.append(two_circles)  

    unique_connections = []
    for pair in range(len(connected_circles)):
        if connected_circles[pair] in unique_connections:
            continue
        if len(connected_circles[pair]) == 0: 
            continue
        else:
            unique_connections.append(connected_circles[pair])

    print(unique_connections)

    # Write connections to a text file
    with open('EdgesAndNodes.txt', 'w') as file:
        file.write(f"{len(circles[0])}\n")
        for connection in unique_connections:
            file.write(f"{connection[0]} {connection[1]}\n")
